In [174]:
import pandas as pd
import numpy as np

In [175]:
user_df=pd.read_csv("E:\\Projects\\Machine Learning\\Music Recommender System\\User Listening History.csv")
user_df.head(3)

,track_id,user_id,playcount
0,TRIRLYL128F42539D1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
1,TRFUPBA128F934F7E1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
2,TRLQPQJ128F42AA94F,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1


In [176]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9711301 entries, 0 to 9711300
Data columns (total 3 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   track_id   object
 1   user_id    object
 2   playcount  int64 
dtypes: int64(1), object(2)
memory usage: 222.3+ MB


In [177]:
name_df_path="E:\\Projects\\Machine Learning\\Music Recommender System\\Music Info.csv"
name_df=pd.read_csv(name_df_path)

In [178]:
name_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50683 entries, 0 to 50682
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   track_id             50683 non-null  object 
 1   name                 50683 non-null  object 
 2   artist               50683 non-null  object 
 3   spotify_preview_url  50683 non-null  object 
 4   spotify_id           50683 non-null  object 
 5   tags                 49556 non-null  object 
 6   genre                22348 non-null  object 
 7   year                 50683 non-null  int64  
 8   duration_ms          50683 non-null  int64  
 9   danceability         50683 non-null  float64
 10  energy               50683 non-null  float64
 11  key                  50683 non-null  int64  
 12  loudness             50683 non-null  float64
 13  mode                 50683 non-null  int64  
 14  speechiness          50683 non-null  float64
 15  acousticness         50683 non-null 

In [179]:
ini_df = pd.merge(user_df, name_df[['track_id', 'name']], on='track_id', how='left')
ini_df

,track_id,user_id,playcount,name
0,TRIRLYL128F42539D1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,Nothing From Nothing
1,TRFUPBA128F934F7E1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,Paper Gangsta
2,TRLQPQJ128F42AA94F,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,Soy
3,TRTUCUY128F92E1D24,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,Stacked Actors
4,TRHDDQG12903CB53EE,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,Heaven's Gonna Burn Your Eyes
...,...,...,...,...
9711296,TRBKFKL128E078ED76,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,1,Hailie's Song
9711297,TRYFXPG128E078ECBD,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,1,Forgot About Dre
9711298,TROBUUZ128F4263002,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,1,Paralyzer
9711299,TROEWXC128F148C83E,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,1,What's The Difference


In [180]:
ini_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9711301 entries, 0 to 9711300
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   track_id   object
 1   user_id    object
 2   playcount  int64 
 3   name       object
dtypes: int64(1), object(3)
memory usage: 296.4+ MB


In [181]:
ini_df.isna().sum()

track_id     0
user_id      0
playcount    0
name         0
dtype: int64

In [182]:
ini_df.duplicated().sum()

np.int64(0)

In [183]:
ini_df["playcount"] = np.log1p(ini_df["playcount"])

In [184]:
ini_df["user_idx"] = ini_df["user_id"].astype("category").cat.codes
ini_df["track_idx"] = ini_df["track_id"].astype("category").cat.codes

In [185]:
import torch
users = torch.tensor(ini_df["user_idx"].values, dtype=torch.long)
tracks = torch.tensor(ini_df["track_idx"].values, dtype=torch.long)
ratings = torch.tensor(ini_df["playcount"].values, dtype=torch.float32)

In [186]:
import torch.nn as nn
import torch.optim as optim

class NCF(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=32):
        super(NCF, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        
        # Fully connected layers
        self.fc1 = nn.Linear(embedding_dim * 2, 64)
        self.fc2 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
    
    def forward(self, user_ids, item_ids):
        user_embeds = self.user_embedding(user_ids)
        item_embeds = self.item_embedding(item_ids)
        x = torch.cat([user_embeds, item_embeds], dim=1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x.squeeze()


In [187]:
user_to_index = {user_id: idx for idx, user_id in enumerate(user_df["user_id"].unique())}
track_to_index = {track_id: idx for idx, track_id in enumerate(name_df["track_id"].unique())}

num_users = len(user_to_index)
num_tracks = len(track_to_index)
model = NCF(num_users, num_tracks)

# Define loss function and optimizer
criterion = nn.MSELoss()  # Since we predict playcounts
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


NCF(
  (user_embedding): Embedding(962037, 32)
  (item_embedding): Embedding(50683, 32)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
)

In [188]:
import itertools

def print_first_10_keys(dictionary):
    """Prints the first 10 keys of a dictionary."""
    first_10_keys = dict(itertools.islice(dictionary.items(), 10)).keys()
    for key in first_10_keys:
        print(key)

print_first_10_keys(user_to_index)

b80344d063b5ccb3212f76538f3d9e43d87dca9e
85c1f87fea955d09b4bec2e36aee110927aedf9a
bd4c6e843f00bd476847fb75c47b4fb430a06856
969cc6fb74e076a68e36a04409cb9d3765757508
4bd88bfb25263a75bbdd467e74018f4ae570e5df
e006b1a48f466bf59feefed32bec6494495a4436
9d6f0ead607ac2a6c2460e4d14fb439a146b7dec
9bb911319fbc04f01755814cb5edb21df3d1a336
b64cdd1a0bd907e5e00b39e345194768e330d652
17aa9f6dbdf753831da8f38c71b66b64373de613


In [189]:
epochs = 10
batch_size = 512
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    predictions = model(users.to(device), tracks.to(device))
    loss = criterion(predictions, ratings.to(device))
    
    loss.backward()
    optimizer.step()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")


Epoch 1/10, Loss: 1.078932762145996
Epoch 2/10, Loss: 1.029823660850525
Epoch 3/10, Loss: 0.9824989438056946
Epoch 4/10, Loss: 0.9369860887527466
Epoch 5/10, Loss: 0.8933089375495911
Epoch 6/10, Loss: 0.8514864444732666
Epoch 7/10, Loss: 0.8115336894989014
Epoch 8/10, Loss: 0.7734612822532654
Epoch 9/10, Loss: 0.7372755408287048
Epoch 10/10, Loss: 0.7029780149459839


In [190]:
import json

# Create mappings
user_to_index = {user_id: idx for idx, user_id in enumerate(user_df["user_id"].unique())}
track_to_index = {track_id: idx for idx, track_id in enumerate(name_df["track_id"].unique())}
index_to_track = {idx: track_id for track_id, idx in track_to_index.items()}
track_to_name = {row["track_id"]: row["name"] for _, row in name_df.iterrows()}

# Save mappings as JSON files
def save_json(data, filename):
    with open(filename, "w") as f:
        json.dump(data, f)

save_json(user_to_index, "user_to_index.json")
save_json(track_to_index, "track_to_index.json")
save_json(index_to_track, "index_to_track.json")
save_json(track_to_name, "track_to_name.json")

print("Mappings saved successfully.")


Mappings saved successfully.


In [191]:
def recommend_songs(user_id, top_n=10):
    if user_id not in user_to_index:
        return f"User ID {user_id} not found in dataset."

    model.eval()
    user_tensor = torch.tensor([user_to_index[user_id]] * len(track_to_index), dtype=torch.long).to(device)
    track_tensor = torch.tensor(list(track_to_index.values()), dtype=torch.long).to(device)

    with torch.no_grad():
        predictions = model(user_tensor, track_tensor)

    _, indices = torch.topk(predictions, top_n)
    recommended_track_ids = [index_to_track[idx.item()] for idx in indices]

    # Convert track IDs to song names
    recommended_songs = [track_to_name.get(track_id, "Unknown Song") for track_id in recommended_track_ids]

    return recommended_songs

In [192]:
print("Embedding Size in Model:", model.item_embedding.num_embeddings)

Embedding Size in Model: 50683


In [195]:
user_id = "e006b1a48f466bf59feefed32bec6494495a4436"
print(recommend_songs(user_id))

['Thick Stew', 'These Are The Days', 'Living Life', 'Like a Prayer', 'Smalltown Boy', 'Generation', 'The Honesty of Constant Human Error', 'The Light That Failed', 'Buried', 'No Other One']


In [194]:
torch.save(model.state_dict(), "collaborative_model.pth")